In [1]:
import pandas as pd
import numpy as np
import os
from os import getcwd

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(getcwd()+"\\train.csv")

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels = df['label'].values

In [5]:
data = df.drop(['label'], axis=1).values

In [6]:
data.shape

(42000, 784)

In [7]:
print(max(data[0]))

255


In [8]:
data = data/255.

In [9]:
print(max(data[0]))

1.0


In [10]:
data = data.reshape((data.shape[0], 28, 28, 1))

In [11]:
data.shape

(42000, 28, 28, 1)

In [12]:
# stack 3 layers of the greyscale image to replicate RGB images
# need to do so in order to use InceptionV3
data_rgb = np.concatenate((data, data, data), axis=-1)

In [13]:
data_rgb.shape

(42000, 28, 28, 3)

In [14]:
data_rgb_exp = tf.image.resize_with_pad(data_rgb, 196, 196)

In [15]:
data_rgb_exp.shape

TensorShape([42000, 196, 196, 3])

In [16]:
data_rgb_exp = np.array(data_rgb_exp)

In [21]:
data_rgb_exp.shape

(42000, 196, 196, 3)

In [17]:
# for row in data_rgb_exp:
#     print(max(row[90, :, 0]))
#     break

In [18]:
df['label'].unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6], dtype=int64)

In [19]:
labels_cat = to_categorical(labels)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data_rgb_exp, labels_cat, test_size=0.3, random_state=101)

In [22]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
# local_weights_file = path_inception

pre_trained_model = InceptionV3(input_shape=(196, 196, 3),
                               include_top=False,
                               weights='imagenet')

In [23]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [24]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 196, 196, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 97, 97, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 97, 97, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 97, 97, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [25]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Expected Output:
# ('last layer output shape: ', (None, 7, 7, 768))

last layer output shape:  (None, 10, 10, 768)


In [27]:
X = tf.keras.layers.Flatten()(last_output)


X = tf.keras.layers.Dense(units=128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)

X = tf.keras.layers.Dense(units=10, activation='softmax')(X)

model = tf.keras.models.Model(pre_trained_model.input, X)

model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['acc'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 196, 196, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 97, 97, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 97, 97, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 97, 97, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [29]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)

Train on 29400 samples, validate on 12600 samples
Epoch 1/10
  224/29400 [..............................] - ETA: 51:22 - loss: 0.9175 - acc: 0.7135

KeyboardInterrupt: 